In [140]:
%pylab inline
import glob
import os

Populating the interactive namespace from numpy and matplotlib


/home/mike/miniconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['sample']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
#figure(figsize=(20, 20))
for sample in sorted(glob.glob('../ABC-GAN/samples_abcgan_nasa_test_3/*'), key=lambda x: os.path.getctime(x))[-10:]:
    figure(figsize=(10, 10))
    print(sample, os.path.getctime(sample))
    im = imread(sample)
    print(im.shape)
    imshow(im)
    

In [11]:
sample_face_img = '../ABC-GAN/data/celebA/000005.jpg'
sample_face_img = imread(sample_face_img)

In [14]:
import img_utils

In [16]:
img_utils.transform(sample_face_img, 64, 64).shape

(64, 64, 3)

In [ ]:
sample_space_img = '../ABC-GAN/data/nasa_test_1/4076.jpg'
sample_space_img = imread(sample_space_img)
imshow(sample_space_img)

In [52]:
sample_space_img.shape

(427, 428, 3)

In [40]:
import glob
import shutil
import os
for impath in glob.glob('../ABC-GAN/data/nasa_test_1/*'):
    im = img_utils.get_image(impath, 64, 64)
    if len(np.array(im).astype(np.float32).shape) != 3:
        print(impath)
        os.remove(impath)

In [27]:
samples = np.array([img_utils.get_image(impath, 64, 64) for impath in glob.glob('../ABC-GAN/data/nasa_test_1/*')])

ValueError: could not broadcast input array from shape (64,64,3) into shape (64,64)

In [41]:
! ls -la ../ABC-GAN/data/nasa_test_1/ | wc -l

106


In [151]:
import json
metas = []
for file in glob.glob('./hubble/meta/*.json'):
    with open(file, 'r', encoding='utf8') as mf:
        try:
            meta = json.load(mf)
            metas.append(meta)
        except:
            print('failed', file)

failed ./hubble/meta/1.json


In [152]:
len(metas), len(list(m for m in metas if 'is_good' in m)), len(list(m for m in metas if 'is_good' in m and m['is_good'] == '1' and len(m['shape'] or []) == 3))

(4149, 2234, 953)

In [153]:
good_pics = list(m for m in metas if 'is_good' in m and m['is_good'] == '1' and len(m['shape'] or []) == 3)

In [154]:
import shutil
import img_utils
import skimage.transform

def apply_transform(image, size=512):
    sx, sy, _ = image.shape
    if sx > sy:
        scale = size / sy
    else:
        scale = size / sx
    
    image = skimage.transform.rescale(image, scale, multichannel=True, mode='constant', anti_aliasing=True)
    return image

def copy_pic(meta, data_dir, transform=True, force=True, size=512):
    if not os.path.isdir(data_dir):
        os.mkdir(data_dir)
    impath = './hubble/jpgs/{}.jpg'.format(meta['id'])
    to_path = os.path.join(data_dir, '{}.jpg'.format(meta['id']))
    if not force and os.path.isfile(to_path):
        return
    
    if transform:
        im = imread(impath)
        im = apply_transform(im, size=size)
        imsave(to_path, im)
    else:
        shutil.move(impath, to_path)

In [138]:
from tqdm import tqdm_notebook as tqdm

for m in tqdm(good_pics):
    copy_pic(m, '../ABC-GAN/data/nasa_test_64/', force=False, size=64)


/home/mike/miniconda3/lib/python3.6/site-packages/PIL/Image.py:2575: DecompressionBombWarning: Image size (128000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


KeyboardInterrupt: 

In [148]:
for m in tqdm(good_pics):
    copy_pic(m, '../ABC-GAN/data/nasa_test_2/', force=False)

/home/mike/miniconda3/lib/python3.6/site-packages/PIL/Image.py:2575: DecompressionBombWarning: Image size (93046800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [158]:
def for_pool(meta):
    copy_pic(meta, '../ABC-GAN/data/nasa_test_64/', force=False, size=64)

In [129]:
from multiprocessing import Pool

In [159]:
with Pool(3) as pool:
    list(tqdm(pool.imap_unordered(for_pool, good_pics, chunksize=2), total=len(good_pics)))

/home/mike/miniconda3/lib/python3.6/site-packages/PIL/Image.py:2575: DecompressionBombWarning: Image size (128000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [135]:
pool.terminate()

In [136]:
pool.join()